In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='dse-staff')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [2]:
import time
import sys
from utils2 import *
sys.path.append('../src')
from config import *

In [ ]:
nd = NDVIPipeline()

polygon = load_protected_area('916').geometry()
buffered_geom = nd.buffer_polygon(polygon, 10000)
water_masked_geom = nd.mask_water(buffered_geom)
col = (ee.ImageCollection('MODIS/061/MOD09A1')
               .filterDate('2001-01-01', '2010-12-31').filterBounds(water_masked_geom))

col1 = map(col.np.mask_modis_clouds)

monthly = nd.monthly_median(modis_col)
monthly = nd.rolling_median(monthly, window=3)
annual = nd.annual_median(monthly)
       # harmonic_results = None
       # if apply_harmonic:
       #     harmonic_results = self.harmonic_modeling(monthly)

        # 9. Extract NDVI time series for water-masked buffered polygon
ndvi_ts = nd.extract_time_series(annual, water_masked_geom)

ndvi_ts

AttributeError: 'ImageCollection' object has no attribute 'mask_modis_clouds'

In [ ]:
# ========== Example usage ===========

# Load a single polygon geometry (example)
# Replace this with your actual polygon feature or geometry

start = time.time()

polygon = load_protected_area('916').geometry()
pipeline = NDVIPipeline()
result = pipeline.run_pipeline(polygon, buffer_distance=10000,
                               apply_smoothing=True, apply_harmonic=False)

print("Biome:", result['biome'])
print(result['ndvi_timeseries'])

end = time.time()
print(f"Total elapsed time: {end - start:.2f} seconds")
# You can now export, plot, or analyze result['ndvi_timeseries']


In [ ]:
import pandas as pd

# Example time series DataFrame
# Index is a datetime or a column named 'year'
df = pd.DataFrame({
    'year': range(1980, 2021),
    'value': [i + (i % 5) for i in range(1980, 2021)]  # sample values
})
df = df.set_index('year')


In [ ]:
def rolling_mean(df, column='value', window=10):
    """
    Computes rolling mean over a specified window.
    
    Args:
        df (pd.DataFrame): Time series DataFrame with index as year or datetime.
        column (str): Column to compute the rolling mean on.
        window (int): Rolling window size in years.
    
    Returns:
        pd.Series: Rolling mean.
    """
    return df[column].rolling(window=window, min_periods=1).mean()


In [ ]:
def rolling_std_mean(df, column='value', window=10):
    """
    Computes rolling mean of the standard deviation over a specified window.
    
    Args:
        df (pd.DataFrame): Time series DataFrame with index as year or datetime.
        column (str): Column to compute the rolling std on.
        window (int): Rolling window size in years.
    
    Returns:
        pd.Series: Rolling mean of standard deviation.
    """
    return df[column].rolling(window=window, min_periods=1).std().rolling(window=window, min_periods=1).mean()
